[Kaggle notebook](https://www.kaggle.com/code/pmassouf/hotel-price-prediction/notebook)

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.model_selection import  cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
import aoai

In [8]:
%config Completer.use_jedi = False

In [5]:
__dataset_object = aoai.Dataset()
__dataset_object.list("Hotel")

C:\Users\91930\Documents\GITHUB\ArtOfAI\dataset\kaggle\hotel_booking_dataset\HotelFinalDataset.xlsx


In [6]:
__dataset_path = __dataset_object.get("Hotel")[0]

In [9]:
df = pd.read_excel(__dataset_path, sheet_name="HotelFinalDataset.xlsx")

In [10]:
df.head()

,Unnamed: 0,Name,Place,Type,Price,ReviewsCount,Rating,City,State
0,0,BUNK Hotel Amsterdam,"Amsterdam Noord, Amsterdam",Bunk Pod for 2,"₹ 7,887",778.0,8.4,Amsterdam Noord,Amsterdam
1,1,YOTEL Amsterdam,"Amsterdam Noord, Amsterdam",Premium Double Room,"₹ 15,267",500.0,8.1,Amsterdam Noord,Amsterdam
2,2,Multatuli Hotel,"Amsterdam City Center, Amsterdam",Double Room,"₹ 13,063",1605.0,7.4,Amsterdam City Center,Amsterdam
3,3,nhow Amsterdam Rai,"Zuideramstel, Amsterdam",nhow Double or Twin Room with View,"₹ 12,854",500.0,9.0,Zuideramstel,Amsterdam
4,4,Motel One Amsterdam,"Zuideramstel, Amsterdam",Double Room,"₹ 9,471",500.0,8.8,Zuideramstel,Amsterdam


We can remove the first column as it is the same as the index of the dataset. The place column is also redundant with the City and State columns

In [11]:
df = df.drop(["Unnamed: 0", "Place"], axis=1)

In [12]:
df.head()

,Name,Type,Price,ReviewsCount,Rating,City,State
0,BUNK Hotel Amsterdam,Bunk Pod for 2,"₹ 7,887",778.0,8.4,Amsterdam Noord,Amsterdam
1,YOTEL Amsterdam,Premium Double Room,"₹ 15,267",500.0,8.1,Amsterdam Noord,Amsterdam
2,Multatuli Hotel,Double Room,"₹ 13,063",1605.0,7.4,Amsterdam City Center,Amsterdam
3,nhow Amsterdam Rai,nhow Double or Twin Room with View,"₹ 12,854",500.0,9.0,Zuideramstel,Amsterdam
4,Motel One Amsterdam,Double Room,"₹ 9,471",500.0,8.8,Zuideramstel,Amsterdam


We can see that we don't seem to have many features to work with. We will have to create more.

### Data preparation and Feature Engineering

In [16]:
df.isna().sum()

Name              0
Type              0
Price             0
ReviewsCount     13
Rating           10
City              0
State           392
dtype: int64

There are NaN values in multiple columns. We will remove rows with NaN values in ReviewsCount and Rating. For state we'll use one hot vectors